# **Parte III: Pico CTF**
### *Xavier Tandazo*
___

#### **Exercise 1: Custom Cyclical Cipher (C3)**

**Description**

We found a brand new type of encryption, can you break the secret code? (Wrap with picoCTF{})

**Explicación:**

La tarea nos da 2 archivos: *cyphertext* y *convert.py*. El script de python toma el *cyphertest* y, carácter por carácter, convierte cada carácter usando dos tablas, ***lookup1 y lookup2***, pero depende del carácter anterior mediante una variable *prev*. 

Lo que se hace es que se almacena todo el texto en variables ***chars***, la variable **prev** se inicializa con 0, y una por una, primero se busca el carácter en la tabla ***lookup1***, esta variable se llama **cur**; posteriormente lo que se hace es identificar ese carácter en la tabla ***lookup2*** y entonces se aplica la siguiente función:
$$
(cur - prev) mod 40
$$
El resultado, es la variable que se llama **out**. Se actualiza el valor de **prev**, haciéndolo igual a **cur** y se continúa el proceso con el siguiente carácter.

Sabemos que la fórmula esencial de este algoritmo es:
$$
k=(cur - prev)mod40
$$
Entonces si sabemos que **k(out)** es el índice que ahora tenemos, es decir el caractér de ***lookup2***, es necesario despejarlo para recuperar el **cur**. Con esto luego actualizaríamos el **prev** y con eso iríamos de carácter en carácter decifrando el mensaje:
$$
cur=(k + prev)mod40
$$

In [9]:
cipher = "DLSeGAGDgBNJDQJDCFSFnRBIDjgHoDFCFtHDgJpiHtGDmMAQFnRBJKkBAsTMrsPSDDnEFCFtIbEDtDCIbFCFtHTJDKerFldbFObFCFtLBFkBAAAPFnRBJGEkerFlcPgKkImHnIlATJDKbTbFOkdNnsgbnJRMFnRBNAFkBAAAbrcbTKAkOgFpOgFpOpkBAAAAAAAiClFGIPFnRBaKliCgClFGtIBAAAAAAAOgGEkImHnIl"
lookup1 = "\n \"#()*+/1:=[]abcdefghijklmnopqrstuvwxyz"
lookup2 = "ABCDEFGHIJKLMNOPQRSTabcdefghijklmnopqrst"

out = ""
prev = 0
for ch in cipher:
    k = lookup2.index(ch) #este es el índice en lookup2 del carácter cifrado
    cur = (k + prev) % 40 #formula invertida
    out += lookup1[cur] #recuperamos el carácter original
    prev = cur

print(out)

#asciiorder
#fortychars
#selfinput
#pythontwo

chars = ""
from fileinput import input
for line in input():
    chars += line
b = 1 / 1

for i in range(len(chars)):
    if i == b * b * b:
        print chars[i] #prints
        b += 1 / 1



El resultado es un script de python2, así que lo corremos en la página *https://onecompiler.com/python2/43xwh9hn3* y obtendremos la flag que es:
### "adlibs"

___
#### **Exercise 2: Easy Peasy (OTP reuse)**

**Explicación**

El ejercicio nos da un servicio OTP remoto (nc mercury.picoctf.net 58913) y el código otp.py.

El servidor imprime primero la **bandera cifrada** usando una clave ***key*** y un índice interno ***key_location*** que avanza después de cada cifrado. Esto nois permite enviar texto para cifrar; para cada petición se hace lo siguiente:

- Tomamos *start = key_location* y *stop = key_location + len(ui)*,
- Extraemos la porción de clave ***key[start:stop]*** y XOR‑ea byte a byte con lo que enviaste ***ui***,
- Actualizamos ***key_location = stop***.

Cuando ***stop >= KEY_LEN*** la implementación hace ***stop = stop % KEY_LEN*** y construye la porción de clave como ***kf[start:] + kf[:stop]***. Es decir, **la clave "da la vuelta"** (wrap‑around) y se vuelve a usar desde el inicio.

Si la misma porción de ***key*** cifra dos mensajes diferentes, los bytes de la clave se cancelan si XOReas los dos cifrados. Entonces:

$$C1 = P1 XOR K$$
$$C2 = P2 XOR K$$
$$C1 XOR C2 = P1 XOR P2$$

Entonces si conocemos *P2*, entonces *P1 = (C1 XOR C2) XOR P2*. Es decir: con un texto conocido podemos recuperar el otro.

Al conectarnos, el servidor ya nos da *C_flag* (el cifrado de la bandera), entonces lo guardamos.  

Si hacemos que *key_location* vuelva al principio y le pedimos al servidor que cifre un texto **conocido** de la **misma longitud** que la bandera, entonces ese texto conocido será cifrado con exactamente la misma porción de clave que se usó para la bandera.  

De tal manera que con *C_flag* y *C_known* (los dos cifrados) podemos calcular la bandera con la fórmula:
  $$
  FLAG = (C_flag XOR C_known) XOR KNOWN
  $$
  donde *KNOWN* es el texto que nosotros enviamos.


Entonces debemos conectarnos y copiar el *C_flag* que imprime el servidor. Posteriormente, calculamos cuánto padding enviar para que *key_location* vuelva al inicio; si la bandera ocupa *F* bytes, enviar *KEY_LEN - F* bytes de relleno hará que la siguiente petición use la clave desde el byte 0.

Ahora, se envía un mensaje conocido de longitud *F* y se copia *C_known*. Calculamos *FLAG_bytes = C_flag XOR C_known XOR b'B'*F* y lo convertiremos a texto; esto nos dará la representación de la flag, en hex por ejemplo.

Sabemos que un **OTP** sólo es seguro si cada byte de clave se usa ***una sola vez***. Aquí la implementación reusa la clave (wrap), así la seguridad se rompe y podemos recuperar la bandera con un texto conocido.


**Código:**

Ejecutaremos el script contenido en la carpeta: *C:\Users\xavie\Downloads\Deber1-Security\PT3\EasyPeasy\exploit_easy_peasy.py*

y esto nos dara la siguiente impresion en la parte final de la terminal, esa será la flag que buscamos para PicoCTF:

#### "...35ecb423b3b43472c35cc2f41011c6d2"

___
#### **Exercise 3: New Caesar**

**Description**

We found a brand new type of encryption, can you break the secret code? (Wrap with picoCTF{})

**Explicación**

El script **news_caesar.py** implementa un nuevo tipo de César, pero sobre un alfabeto de 16. Entonces, el mensaje original se transforma primero en **binario de 8 bits** por carácter, y luego cada nibble de 4 bits se convierte en una letra del alfabeto reducido.

Se convierte el carácter a su código ASCII:

$$
\text{byte} = \text{ord}(\text{plain})
$$

Se separa en dos nibbles de 4 bits:

$$
\text{hi} = \left\lfloor \frac{\text{byte}}{16} \right\rfloor, \quad 
\text{lo} = \text{byte} \bmod 16
$$

Para cada nibble se codifica en una letra del alfabeto reducido:

$$
\text{b16} = ALPHABET[\text{hi}] \, ALPHABET[\text{lo}]
$$

El cifrado final aplica un desplazamiento \(k\) módulo 16 a cada letra codificada:

$$
c = (b + k) \bmod 16
$$

donde:  
- \(b\) es la posición de la letra en el alfabeto reducido a..p  
- \(c\) es la letra cifrada  
- \(k\) es la clave (un solo carácter)

De esta forma entonces tenemos:

#### "apbopjbobpnjpjnmnnnmnlnbamnpnononpnaaaamnlnkapndnkncamnpapncnbannaapncndnlnpna"

Ahora para revertir el cifrado:

Revertimos el desplazamiento César

$$
b = (c - k) \bmod 16
$$

Ahora los agrupamos en pares y reconstruimos el byte original:

$$
\text{byte} = (\text{hi} \times 16) + \text{lo}
$$

donde:

$$
\text{hi} = \text{ALPHABET.index}(b_{}), \quad
\text{lo} = \text{ALPHABET.index}(b_{})
$$

Convertimos el byte a ASCII:

$$
\text{plain} = \text{chr}(\text{byte})
$$

De esta forma, concatenando todos los caracteres obtenemos el **mensaje original**. Y este proceso se repetiría hasta reconstruir todo el mensaje y obtener la flag.


In [ ]:
import string 
ALPHABET = string.ascii_lowercase[:16]  #alfabeto de 16 letras

ciphertext = "apbopjbobpnjpjnmnnnmnlnbamnpnononpnaaaamnlnkapndnkncamnpapncnbannaapncndnlnpna"

def unshift(c, k):
    t1 = ord(c) - ord('a')  #calculamos la posicion de la letra cifrada en base a a
    t2 = ord(k) - ord('a')  #calculamos la posicion de la clave en base a a
    return ALPHABET[(t1 - t2) % len(ALPHABET)]  # e calcula el desplazamiento inverso modulo 16

#funcion para decodificar b16 a ascii
def b16_decode(enc):
    plain = ""  
    for i in range(0, len(enc), 2):  #analizamos el texto de dos en dos letras
        hi = ALPHABET.index(enc[i])  #calculamos el nibble alto a partir de la primera letra
        lo = ALPHABET.index(enc[i+1])  #calculamos el nibble bajo a partir de la segunda letra
        byte = (hi << 4) | lo  #ahora reconstruimos el byte original combinando hi y lo
        plain += chr(byte)  #y convertimos el byte a caracter ascii y concatenamos
    return plain

#lista de candidatos
candidates = []  #almacenamos los resultados de cada clave

#todas las posibles claves del alfabeto
for k in ALPHABET:
    unshifted = "".join(unshift(c, k) for c in ciphertext) 
    try:
        flag = b16_decode(unshifted) 
    except Exception:
        continue  

    pr = sum(1 for ch in flag if ch.isprintable()) / len(flag)

    hexlen = max((len(r) for r in re.findall(r'[0-9a-fA-F]{4,}', flag)), default=0)

    letters = sum(1 for ch in flag if ch.isalpha())
    candidates.append((k, flag, pr, hexlen, letters))  #almacenamos los valores calculados para cada clave

#ordenamos los candidatos por proporcion de imprimibles, longitud de secuencia hex y cantidad de letras
candidates.sort(key=lambda x: (x[2], x[3], x[4]), reverse=True)

# imprimimos solo la mejor opcion
if candidates:
    best_k, best_flag, best_pr, best_hexlen, _ = candidates[0] 
    print("Mejor k:", best_k)
    print("Resultado:", best_flag)  
else:
    print("No hay candidatos válida.") 


Mejor k: k
Resultado: et_tu?_23217b54456fb10e908b5e87c6e89156


Entonces la flag es:

#### et_tu?_23217b54456fb10e908b5e87c6e89156

___
#### **Exercise 4: Rotation**

**Description**

You will find the flag after decrypting this file:

#### xqkwKBN{z0bib1wv_l3kzgxb3l_i4j7l759}

**Explicación**

Este ejercicio se basa en un cifrado por **rotación** en el que cada letra del texto se desplaza un número fijo de posiciones a lo largo del alfabeto. Tal y como indica el título del enunciado, y teniendo en cuenta que la pista dice: "*Sometimes rotation is right*", el reto es **descifrar el mensaje aplicando la rotación inversa correcta** para recuperar la flag.

Observando el texto cifrado *xqkwKBN{z0bib1wv_l3kzgxb3l_i4j7l759}*, notamos que solo las letras están desplazadas, mientras que los símbolos permanecen intactos. De entrada, notamos que ya nos está dando el formato de una flag para pico: "picoCTF{...}"

Como el alfabeto latino tiene 26 letras, existen **26 posibles desplazamientos**. Para cada desplazamiento, se aplica la rotación inversa a cada letra del texto cifrado:
   
Letras minúsculas:  
     $$
     c_i = (c_\text{orig} - n) \bmod 26
     $$
   
Letras mayúsculas:  
     $$
     C_i = (C_\text{orig} - n) \bmod 26
     $$


Entonces, se revisarían todos los resultados de todas las rotaciones y se identifica la que genera un texto legible y que contenga la estructura de flag. En este caso, la rotación correcta produce:


In [ ]:
def rot(s, n):
    res = []
    for ch in s:
        if 'a' <= ch <= 'z':
            res.append(chr((ord(ch) - ord('a') + n) % 26 + ord('a')))
        elif 'A' <= ch <= 'Z':
            res.append(chr((ord(ch) - ord('A') + n) % 26 + ord('A')))
        else:
            res.append(ch)
    return ''.join(res)

cipher = "xqkwKBN{z0bib1wv_l3kzgxb3l_i4j7l759}"

for shift in range(26):
    candidate = rot(cipher, shift)
    if "picoCTF{" in candidate:
        print("shift:", shift)
        print(candidate)
        break


shift: 18
picoCTF{r0tat1on_d3crypt3d_a4b7d759}


Entonces la flag es:

#### r0tat1on_d3crypt3d_a4b7d759

___
#### **Exercise 5: Vigenere**

**Description**

Can you decrypt this message?:

#### rgnoDVD{O0NU_WQ3_G1G3O3T3_A1AH3S_f85729e7}

Decrypt this message using this key "CYLAB".

**Explicación**

Sabemos que el cifrado de Vigenère es un cifrado por sustitución **polialfabético** que utiliza una **clave repetida** para cifrar un mensaje. 

Entonces cada letra se convierte en un número del 0 al 25:

$$
A = 0, \; B = 1, \; C = 2, \dots, Z = 25
$$

Para cifrar un mensaje \(m\) usando una clave \(k\) de longitud \(|k|\):

1. Repetimos la clave tantas veces como sea necesario para cubrir todo el mensaje.
2. Para cada letra \(m\) del mensaje, sumamos el valor de la letra correspondiente de la clave \(k_j\):

$$
c = (m + k_{i \bmod |k|}) \bmod 26
$$

Donde:  
- \(c\) es la letra cifrada,  
- \(m\) es la letra original del mensaje,  
- \(k_{i \bmod |k|}\) es la letra de la clave correspondiente.


Entonces si queremos recuperar el mensaje original, solo despejamos la m:

$$
m = (c - k_{i \bmod |k|}) \bmod 26
$$

Es necesario saber que los caracteres no alfabéticos **no se cifran** y se mantienen igual.


In [11]:
def vigenere_decrypt(ciphertext, key):
    plaintext = ""
    key_indices = [ord(k.upper()) - ord('A') for k in key]  #convertimos la clave a números 0-25
    key_len = len(key)
    key_idx = 0  #indice para recorrer la clave

    for ch in ciphertext:
        if ch.isupper():  #mayúsculas
            c = ord(ch) - ord('A')  #convertimos la letra cifrada a número
            m = (c - key_indices[key_idx % key_len]) % 26  #aplicamos la fórmula m
            plaintext += chr(m + ord('A'))
            key_idx += 1
        elif ch.islower():  #letras minúsculas
            c = ord(ch) - ord('a')
            m = (c - key_indices[key_idx % key_len]) % 26
            plaintext += chr(m + ord('a'))
            key_idx += 1
        else:  #caracteres no alfabéticos se dejan igual
            plaintext += ch

    return plaintext

ciphertext = "rgnoDVD{O0NU_WQ3_G1G3O3T3_A1AH3S_f85729e7}"
key = "CYLAB"

flag = vigenere_decrypt(ciphertext, key)
print(flag)

picoCTF{D0NT_US3_V1G3N3R3_C1PH3R_d85729g7}


Entonces la flag es:

#### D0NT_US3_V1G3N3R3_C1PH3R_d85729g7